In [11]:
using LinearAlgebra, Distributions, Combinatorics, Random, Kronecker, SpecialFunctions
using ResumableFunctions
include("../DCM_model/BBVI_utils.jl")

# Object for observed data
struct TDCMObs{T <: AbstractFloat}
    # data
    Y           :: Array{Int, 3}
    Q           :: Matrix{Int}
    D           :: Vector{Matrix{Int}}
    U           :: Vector{Vector{Matrix{T}}}
    X           :: Vector{Vector{Matrix{T}}}
    group       :: Vector{Int}
    skill_dict  :: Dict{Int, Vector{Int}}
end

function TDCMObs(
    Y       :: Array{Int, 3}, 
    Q       :: Matrix{Int},
    U       :: Vector{Vector{Matrix{T}}},
    X       :: Vector{Vector{Matrix{T}}},
    group   :: Vector{Int}) where T <: AbstractFloat
    D = generate_delta(Q)
    K, L = size(Q, 2), size(D[1], 1)
    skill_dict = Dict{Int, Vector{Int}}()
    for l in 0:(L - 1)
        skill_dict[l + 1] = reverse(digits(l, base=2, pad=K))
    end
    TDCMObs(Y, Q, D, U, X, group, skill_dict)
end

# Object including latent variables and model parameters
struct TDCModel{T <: AbstractFloat}
    # data
    obs         :: TDCMObs
    # prior distribution parameters
    mu_beta_prior       :: Vector{Vector{T}}
    V_beta_prior        :: Vector{Matrix{T}}
    mu_omega_prior      :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_omega_prior       :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    a_tau_prior         :: Vector{Vector{Vector{Vector{T}}}}
    b_tau_prior         :: Vector{Vector{Vector{Vector{T}}}}
    # This option allocates extra memory based on number of threads availible in the environment
    enable_parallel     :: Bool
    # Variational distribution parameters
    pi_star             :: Vector{Vector{Vector{T}}}
    mu_beta_star        :: Vector{Vector{T}}
    V_beta_star         :: Vector{Matrix{T}}
    mu_gamma_star       :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_gamma_star        :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    mu_omega_star       :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_omega_star        :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    a_tau_star          :: Vector{Vector{Vector{Vector{T}}}}
    b_tau_star          :: Vector{Vector{Vector{Vector{T}}}}
    # Number of samples for noisy gradient
    M                   :: Int
    # Preallocated storage for samples from variational distribution
    Z_sample            :: Vector{Vector{Vector{Vector{Int}}}}
    beta_sample         :: Vector{Vector{Vector{T}}}
    gamma_sample        :: Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}
    omega_sample        :: Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}
    tau_sample          :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    # Preallocated storage for noisy gradient descent calculations
    storage_L           :: Vector{T}
    storage_L2          :: Vector{T}
    storage_L3          :: Vector{T}
    storage_L4          :: Vector{T}
    storage_LL          :: Matrix{T}
    storage_LL2         :: Matrix{T}
    storage_LL3         :: Matrix{T}
    storage_LL4         :: Matrix{T}
    # Preallocated storage for matrix vectorization operations
    storage_comm        :: Matrix{T}
    storage_dup         :: Matrix{T}
    storage_Lsqr        :: Vector{T}
    storage_Lsqr2       :: Vector{T}
    storage_L2L2        :: Matrix{T}
    storage_C           :: Matrix{T}
    storage_gradC       :: Vector{T}
    # Preallocated Identity matrix
    I_LL                :: Matrix{T}
    # Preallocated storage for parallel noisy gradient descent calculations
    storage_L_par       :: Vector{Vector{T}}
    storage_L2_par      :: Vector{Vector{T}}
    storage_L3_par      :: Vector{Vector{T}}
    storage_L4_par      :: Vector{Vector{T}}
    storage_LL_par      :: Vector{Matrix{T}}
    storage_LL2_par     :: Vector{Matrix{T}}
    storage_LL3_par     :: Vector{Matrix{T}}
    storage_LL4_par     :: Vector{Matrix{T}}
    # Preallocated storage for parallel matrix vectorization operations
    storage_comm_par    :: Vector{Matrix{T}}
    storage_dup_par     :: Vector{Matrix{T}}
    storage_Lsqr_par    :: Vector{Vector{T}}
    storage_Lsqr2_par   :: Vector{Vector{T}}
    storage_L2L2_par    :: Vector{Matrix{T}}
    storage_C_par       :: Vector{Matrix{T}}
    storage_gradC_par   :: Vector{Vector{T}}
end

function TDCModel(
    obs                 :: TDCMObs,
    mu_beta_prior       :: Vector{Vector{T}},
    V_beta_prior        :: Vector{Matrix{T}},
    mu_omega_prior      :: Vector{Vector{Vector{Vector{Vector{T}}}}},
    V_omega_prior       :: Vector{Vector{Vector{Vector{Matrix{T}}}}},
    a_tau_prior         :: Vector{Vector{Vector{Vector{T}}}},
    b_tau_prior         :: Vector{Vector{Vector{Vector{T}}}},
    M                   :: Int;
    # This option allocates extra memory based on number of threads availible in the environment
    enable_parallel     :: Bool=false
) where T <: AbstractFloat
    # Number of students, time points, questions, skills, attribute profiles, groups
    N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)
    # Initialize variational distribution parameters
    pi_star = Vector{Vector{Vector{T}}}(undef, N)
    for i in 1:N
        pi_star[i] = Vector{Vector{T}}(undef, O)
        for t in 1:O
            # probability vector of possible single skill mastery over all time points
            pi_star[i][t] = ones(2^K) ./ 2^K
        end
    end
    mu_beta_star = Vector{Vector{T}}(undef, J)
    V_beta_star = Vector{Matrix{T}}(undef, J)
    for j in 1:J
        num_features = size(obs.D[j], 2)
        mu_beta_star[j] = zeros(num_features)
        V_beta_star[j] = Matrix(1.0I, num_features, num_features)
    end
    mu_gamma_star = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, K)
    V_gamma_star = Vector{Vector{Vector{Vector{Matrix{T}}}}}(undef, K)
    for k in 1:K
        mu_gamma_star[k] = Vector{Vector{Vector{Vector{T}}}}(undef, O)
        V_gamma_star[k] = Vector{Vector{Vector{Matrix{T}}}}(undef, O)
        for t in 1:O
            if t == 1
                mu_gamma_star[k][t] = Vector{Vector{Vector{T}}}(undef, 1)
                V_gamma_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 1)

                mu_gamma_star[k][t][1] = Vector{Vector{T}}(undef, S)
                V_gamma_star[k][t][1] = Vector{Matrix{T}}(undef, S)
                for s in 1:S
                    mu_gamma_star[k][t][1][s] = zeros(1)
                    V_gamma_star[k][t][1][s] = ones(1, 1)
                end
            else
                mu_gamma_star[k][t] = Vector{Vector{Vector{T}}}(undef, 2)
                V_gamma_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 2)
                num_features = size(obs.X[k][t], 2)
                for z in 1:2
                    mu_gamma_star[k][t][z] = Vector{Vector{T}}(undef, S)
                    V_gamma_star[k][t][z] = Vector{Matrix{T}}(undef, S)
                    for s in 1:S
                        mu_gamma_star[k][t][z][s] = zeros(num_features)
                        V_gamma_star[k][t][z][s] = Matrix(1.0I, num_features, num_features)
                    end
                end
            end
        end
    end
    mu_omega_star = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, K)
    V_omega_star = Vector{Vector{Vector{Vector{Matrix{T}}}}}(undef, K)
    a_tau_star = Vector{Vector{Vector{Vector{T}}}}(undef, K)
    b_tau_star = Vector{Vector{Vector{Vector{T}}}}(undef, K)
    for k in 1:K
        mu_omega_star[k] = Vector{Vector{Vector{Vector{T}}}}(undef, O)
        V_omega_star[k] = Vector{Vector{Vector{Matrix{T}}}}(undef, O)
        a_tau_star[k] = Vector{Vector{Vector{T}}}(undef, O)
        b_tau_star[k] = Vector{Vector{Vector{T}}}(undef, O)
        for t in 1:O
            num_features_gamma = size(obs.X[k][t], 2)
            num_features_omega = size(obs.U[k][t], 2)
            if t == 1
                mu_omega_star[k][t] = Vector{Vector{Vector{T}}}(undef, 1)
                mu_omega_star[k][t][1] = Vector{Vector{T}}(undef, 1)
                mu_omega_star[k][t][1][1] = zeros(num_features_omega)

                V_omega_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 1)
                V_omega_star[k][t][1] = Vector{Matrix{T}}(undef, 1)
                V_omega_star[k][t][1][1] = Matrix{T}(1.0I, num_features_omega, num_features_omega)

                a_tau_star[k][t] = Vector{Vector{T}}(undef, 1)
                a_tau_star[k][t][1] = ones(1)

                b_tau_star[k][t] = Vector{Vector{T}}(undef, 1)
                b_tau_star[k][t][1] = ones(1)
            else
                mu_omega_star[k][t] = Vector{Vector{Vector{T}}}(undef, 2)
                V_omega_star[k][t] = Vector{Vector{Matrix{T}}}(undef, 2)
                a_tau_star[k][t] = Vector{Vector{T}}(undef, 2)
                a_tau_star[k][t] = Vector{Vector{T}}(undef, 2)
                for z in 1:2
                    mu_omega_star[k][t][z] = Vector{Vector{T}}(undef, num_features_gamma)
                    V_omega_star[k][t][z] = Vector{Matrix{T}}(undef, num_features_gamma)
                    a_tau_star[k][t][z] = ones(num_features_gamma) .* 3
                    a_tau_star[k][t][z] = ones(num_features_gamma) .* 3
                    for m in 1:num_features_gamma
                        mu_omega_star[k][t][z][m] = zeros(num_features_omega)
                        V_omega_star[k][t][z][m] = Matrix(1.0I, num_features_omega, num_features_omega)
                    end
                end
            end
        end
    end
    # Preallocate space for samples from variational distribution
    Z_sample = Vector{Vector{Vector{Vector{Int}}}}(undef, N)
    for i in 1:N
        Z_sample[i] = Vector{Vector{Vector{Int}}}(undef, O)
        for t in 1:O
            Z_sample[i][t] = Vector{Vector{Int}}(undef, M)
            for m in 1:M
                Z_sample[i][t][m] = Vector{Int}(undef, 2^(K))
            end
        end
    end
    beta_sample = Vector{Vector{Vector{T}}}(undef, J)
    for j in 1:J
        beta_sample[j] = Vector{Vector{T}}(undef, M)
        num_features = size(obs.D[j], 2)
        for m in 1:M
            beta_sample[j][m] = Vector{T}(undef, num_features)
        end
    end
    gamma_sample = Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}(undef, K)
    for k in 1:K
        gamma_sample[k] = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, O)
        for t in 1:O
            if t == 1
                gamma_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 1)
                gamma_sample[k][t][1] = Vector{Vector{Vector{T}}}(undef, S)
                for s in 1:S
                    gamma_sample[k][t][1][s] = Vector{Vector{T}}(undef, M)
                    for m in 1:M
                        gamma_sample[k][t][1][s][m] = Vector{T}(undef, 1)
                    end
                end
            else
                num_features = size(obs.X[k][t], 2)
                gamma_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 2)
                for z in 1:2
                    gamma_sample[k][t][z] = Vector{Vector{Vector{T}}}(undef, S)
                    for s in 1:S
                        gamma_sample[k][t][z][s] = Vector{Vector{T}}(undef, M)
                        for m in 1:M
                            gamma_sample[k][t][z][s][m] = Vector{T}(undef, num_features)
                        end
                    end
                end
            end
        end
    end
    omega_sample = Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}(undef, K)
    tau_sample = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, K)
    for k in 1:K
        omega_sample[k] = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, O)
        tau_sample[k] = Vector{Vector{Vector{Vector{T}}}}(undef, O)
        for t in 1:O
            num_features_gamma = size(obs.X[k][t], 2)
            num_features_omega = size(obs.U[k][t], 2)
            if t == 1
                omega_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 1)
                omega_sample[k][t][1] = Vector{Vector{Vector{T}}}(undef, 1)
                omega_sample[k][t][1][1] = Vector{Vector{T}}(undef, M)

                tau_sample[k][t] = Vector{Vector{Vector{T}}}(undef, 1)
                tau_sample[k][t][1] = Vector{Vector{T}}(undef, 1)
                tau_sample[k][t][1][1] = Vector{T}(undef, M)
                for m in 1:M
                    omega_sample[k][t][1][1][m] = Vector{T}(undef, num_features_omega)
                end
            else
                omega_sample[k][t] = Vector{Vector{Vector{Vector{T}}}}(undef, 2)
                tau_sample[k][t] = Vector{Vector{Vector{T}}}(undef, 2)
                for z in 1:2
                    omega_sample[k][t][z] = Vector{Vector{Vector{T}}}(undef, num_features_gamma)
                    tau_sample[k][t][z] = Vector{Vector{T}}(undef, num_features_gamma)
                    for g in 1:num_features_gamma
                        omega_sample[k][t][z][g] = Vector{Vector{T}}(undef, M)
                        tau_sample[k][t][z][g] = Vector{T}(undef, M)
                        for m in 1:M
                            omega_sample[k][t][z][g][m] = Vector{T}(undef, num_features_omega)
                        end
                    end
                end
            end
        end
    end
    # Preallocate storage for noisy gradient descent calculations
    storage_L = Vector{T}(undef, L)
    storage_L2 = similar(storage_L)
    storage_L3 = similar(storage_L)
    storage_L4 = similar(storage_L)
    storage_LL = Matrix{T}(undef, L, L)
    storage_LL2 = similar(storage_LL)
    storage_LL3 = similar(storage_LL)
    storage_LL4 = similar(storage_LL)
    # Preallocate storage for matrix vectorization operations
    storage_comm = Matrix{T}(undef, L^2, L^2)
    storage_dup = Matrix{T}(undef, L^2, Int(L*(L+1)/2))
    storage_Lsqr = Vector{T}(undef, L^2)
    storage_Lsqr2 = Vector{T}(undef, L^2)
    storage_L2L2 = Matrix{T}(undef, L^2, L^2)
    storage_C = Matrix{T}(undef, L, L)
    storage_gradC = Vector{T}(undef, Int(L*(L+1)/2))
    # Preallocate Identity matrix
    I_LL = Matrix{T}(I, L, L)
    # Allocate optional space for parallel computing
    nthreads = Threads.nthreads()
    storage_L_par = Vector{Vector{T}}(undef, nthreads)
    storage_L2_par = similar(storage_L_par)
    storage_L3_par = similar(storage_L_par)
    storage_L4_par = similar(storage_L_par)
    storage_LL_par = Vector{Matrix{T}}(undef, nthreads)
    storage_LL2_par = similar(storage_LL_par)
    storage_LL3_par = similar(storage_LL_par)
    storage_LL4_par = similar(storage_LL_par)
    storage_comm_par = Vector{Matrix{T}}(undef, nthreads)
    storage_dup_par = Vector{Matrix{T}}(undef, nthreads)
    storage_Lsqr_par = Vector{Vector{T}}(undef, nthreads)
    storage_Lsqr2_par = Vector{Vector{T}}(undef, nthreads)
    storage_L2L2_par = Vector{Matrix{T}}(undef, nthreads)
    storage_C_par = Vector{Matrix{T}}(undef, nthreads)
    storage_gradC_par = Vector{Vector{T}}(undef, nthreads)
    if enable_parallel
        storage_L_par[1] = storage_L
        storage_L2_par[1] = storage_L2
        storage_L3_par[1] = storage_L3
        storage_L4_par[1] = storage_L4
        storage_LL_par[1] = storage_LL
        storage_LL2_par[1] = storage_LL2
        storage_LL3_par[1] = storage_LL3
        storage_LL4_par[1] = storage_LL4
        storage_comm_par[1] = storage_comm
        storage_dup_par[1] = storage_dup
        storage_Lsqr_par[1] = storage_Lsqr
        storage_Lsqr2_par[1] = storage_Lsqr2
        storage_L2L2_par[1] = storage_L2L2
        storage_C_par[1] = storage_C
        storage_gradC_par[1] = storage_gradC_par
        for thread in 2:nthreads
            storage_L_par[thread] = Vector{T}(undef, L)
            storage_L2_par[thread] = similar(storage_L)
            storage_L3_par[thread] = similar(storage_L)
            storage_L4_par[thread] = similar(storage_L)
            storage_LL_par[thread] = Matrix{T}(undef, L, L)
            storage_LL2_par[thread] = similar(storage_LL)
            storage_LL3_par[thread] = similar(storage_LL)
            storage_LL4_par[thread] = similar(storage_LL)
            storage_comm_par[thread] = Matrix{T}(undef, L^2, L^2)
            storage_dup_par[thread] = Matrix{T}(undef, L^2, Int(L*(L+1)/2))
            storage_Lsqr_par[thread] = Vector{T}(undef, L^2)
            storage_Lsqr2_par[thread] = Vector{T}(undef, L^2)
            storage_L2L2_par[thread] = Matrix{T}(undef, L^2, L^2)
            storage_C_par[thread] = Matrix{T}(undef, L, L)
            storage_gradC_par[thread] = Vector{T}(undef, Int(L*(L+1)/2))
        end
        println("TDCModel constructed for computation on $nthreads threads")
    end
    # Initialize DCModel object
    TDCModel(obs, mu_beta_prior, V_beta_prior, mu_omega_prior, V_omega_prior, a_tau_prior, b_tau_prior, enable_parallel,
    pi_star, mu_beta_star, V_beta_star, mu_gamma_star, V_gamma_star, mu_omega_star, V_omega_star, a_tau_star, b_tau_star, M,
    Z_sample, beta_sample, gamma_sample, omega_sample, tau_sample,
    storage_L, storage_L2, storage_L3, storage_L4, storage_LL, storage_LL2, storage_LL3, storage_LL4,
    storage_comm, storage_dup, storage_Lsqr, storage_Lsqr2, storage_L2L2, storage_C, storage_gradC,
    I_LL, storage_L_par, storage_L2_par, storage_L3_par, storage_L4_par, storage_LL_par, storage_LL2_par, storage_LL3_par, storage_LL4_par,
    storage_comm_par, storage_dup_par, storage_Lsqr_par, storage_Lsqr2_par, storage_L2L2_par, storage_C_par, storage_gradC_par)
end

TDCModel

In [5]:
function sample_Z(
    model           :: TDCModel,
    idx_student     :: Int,
    idx_time        :: Int
)
    # Create variational distribution from model parameters for specific Z
    Z_it_variational_distribution = Multinomial(1, model.pi_star[idx_student][idx_time])
    # Populate preallocated arrays with samples from variational distribution
    rand!(Z_it_variational_distribution, model.Z_sample[idx_student][idx_time])
end

function sample_β(
    model           :: TDCModel;
    idx_question    :: Int = -1
)
    obs = model.obs
    J = size(obs.Y, 3)
    if idx_question == -1
        for j in 1:J
            # Create variational distribution from model parameters for each β_j
            beta_j_variational_distribution = MvNormal(model.mu_beta_star[j], model.V_beta_star[j])
            # Populate preallocated arrays with samples from variational distribution
            rand!(beta_j_variational_distribution, model.beta_sample[j])
        end
    else
        # Create variational distribution from model parameters for specific β_j
        beta_j_variational_distribution = MvNormal(model.mu_beta_star[idx_question], model.V_beta_star[idx_question])
        # Populate preallocated arrays with samples from variational distribution
        rand!(beta_j_variational_distribution, model.beta_sample[idx_question])
    end
end

function sample_γ(
    model           :: TDCModel,
    idx_group       :: Int,
    idx_time        :: Int,
    idx_skill       :: Int,
    indicator_skill :: Int
)
    # Create variational distribution from model parameters for γ
    gamma_stkz_variational_distribution = MvNormal(model.mu_gamma_star[idx_skill][idx_time][indicator_skill + 1][idx_group],
                                            model.V_gamma_star[idx_skill][idx_time][indicator_skill + 1][idx_group])
    # Populate preallocated arrays with samples from variational distribution
    rand!(gamma_stkz_variational_distribution, model.gamma_sample[idx_skill][idx_time][indicator_skill + 1][idx_group])
end
;

In [ ]:
# function update_categorical_variational_distribution(
#     params              :: Vector{T},
#     modelloglikelihood_func, 
#     storage_samples     :: Vector{Vector{Int}};
#     step                :: T = 1e-2,
#     maxiter             :: Int = 100000,
#     verbose             :: Bool = true,
#     storage1            :: Vector{T} = nothing,
#     storage2            :: Vector{T} = nothing,
#     storage3            :: Vector{T} = nothing,
#     storage4            :: Vector{T} = nothing
# ) where T <: AbstractFloat
#     # Storage for gradient terms
#     grad_log_q = storage1
#     grad_L = model.storage2
#     if storage_grad1 == nothing
#         grad_log_q = similar(params)
#     end
#     if storage_grad2 == nothing
#         grad_L = similar(params)
#     end
#     # Storage for intermediate term in gradient calculations
#     D_beta = storage3
#     rho_star_old_i = storage4
#     if storage3 == nothing
#         D_beta = similar(params)
#     end
#     if storage4 == nothing
#         rho_star_old_i = similar(params)
#     end
#     M = size(storage_samples, 1)
#     # Perform gradient descent update of i-th π*    
#     @inbounds for iter in 1:maxiter
#         # Rho is unique up to a constant addative term
#         rho_star_old_i = log.(params)
#         # Sample Z with current π*
#         categorical_variational_distribution = Multinomial(1, params)
#         # Populate preallocated arrays with samples from variational distribution
#         rand!(categorical_variational_distribution, storage_samples)
#         # Set gradient of ELBO to 0
#         fill!(grad_L, 0)
#         # Rao Blackwellized ELBO
#         ELBO = 0
#         # Calculate the gradient estimate of the m-th sample
#         @inbounds for m in 1:M
#             z_m = storage_samples[m]
#             # Calculate gradient of log(q_1i(Z_i)) w.r.t. π*_i
#             grad_log_q .= z_m .- params
#             # Calculate log(q_1i(Z_i))
#             log_q = dot(z_m, log.(params))
#             # Get log likelihood of m-th sample
#             log_prob = modelloglikelihood_func(z_m)
#             # Update average gradient
#             grad_L .= (m - 1)/m .* grad_L + 1/m .* grad_log_q .* (log_prob - log_q)
#             # Update ELBO estimator
#             ELBO = (m-1)/m * ELBO + 1/m * (log_prob - log_q)
#         end
#         # Print ELBO, parameter and gradient if verbose
#         if verbose
#             println("ELBO: $ELBO")
#             println("π*_$i: $params")
#             println("gradient: $grad_L")
#         end
#         # Update with one step
#         rho_star_old_i .+= step * grad_L
#         # Convert logits into probabilities
#         params .= exp.(rho_star_old_i) ./ sum(exp.(rho_star_old_i))
#     end
# end

update_categorical_variational_distribution (generic function with 1 method)

In [7]:
function update_categorical_variational_distribution(
    model               :: TDCModel;
    step                :: T = 1e-2,
    tol                 :: T = 1e-6,
    maxiter             :: Int = 100000,
    verbose             :: Bool = true
) where T <: AbstractFloat
    obs = model.obs
    Y, D, X = obs.Y, obs.D, obs.X
    Z_sample, beta_sample, gamma_sample = model.Z_sample, model.beta_sample, model.gamma_sample
    pi_star_old = model.pi_star
    # Number of students, time points, questions, skills, attribute profiles, groups
    N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)
    M = model.M
    # Fully update parameters of each Z_i using noisy gradients before moving to update parameters of next Z_i
    if !model.enable_parallel
        @inbounds for i in 1:N
            # Storage for gradient terms
            grad_log_q = model.storage_L2
            grad_L = model.storage_L3
            # Storage for intermediate term in gradient calculations
            D_beta = model.storage_L
            rho_star_old_i = view(model.storage_LL3, 1:L)
            # Get parameters for variational distribution of skill of i-th student
            pi_star_old_i = pi_star_old[i][1]
            # Get group number of student i
            group_i = obs.group[i]
            # Perform gradient descent update of i-th π*    
            @inbounds for iter in 1:maxiter
                # Rho is unique up to a constant addative term
                rho_star_old_i = log.(pi_star_old_i)
                # Sample Z with updated π*
                sample_Z(model, i, 1)
                # Set gradient of ELBO to 0
                fill!(grad_L, 0)
                # Rao Blackwellized ELBO
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                @inbounds for m in 1:M
                    z_im = Z_sample[i][1][m]
                    # Calculate gradient of log(q_1i(Z_i)) w.r.t. π*_i
                    grad_log_q .= z_im .- pi_star_old_i
                    # Calculate log(p(Y, Z_(i)))
                    log_prob_YZ = 0
                    @inbounds for j in 1:J
                        mul!(D_beta, D[j], beta_sample[j][m])
                        log_prob_YZ += dot(z_im, log.(sigmoid.((2*Y[i, 1, j] - 1) .* D_beta)))
                    end
                    skill_profile = obs.skill_dict[argmax(z_im)]
                    @inbounds for k in 1:K
                        log_prob_YZ += log(sigmoid((2*skill_profile[k] - 1) * dot(gamma_sample[k][1][1][group_i][m], obs.X[k][1][i])))
                    end
                    # Calculate log(q_1i(Z_i))
                    log_q = dot(z_im, log.(pi_star_old_i))
                    # Update average gradient
                    grad_L .= (m - 1)/m .* grad_L + 1/m .* grad_log_q .* (log_prob_YZ - log_q)
                    # Update ELBO estimator
                    ELBO = (m-1)/m * ELBO + 1/m * (log_prob_YZ - log_q)
                end
                # Print ELBO, parameter and gradient if verbose
                if verbose
                    println("ELBO: $ELBO")
                    println("π*_$i: $pi_star_old_i")
                    println("gradient: $grad_L")
                end
                # Update with one step
                rho_star_old_i .+= step * grad_L
                # Convert logits into probabilities
                pi_star_old_i .= exp.(rho_star_old_i) ./ sum(exp.(rho_star_old_i))
                # Stop condition
                if abs2(norm(grad_L)) <= tol
                    break
                end
            end
        end
    else
        Threads.@threads for i in 1:N
            # Get thread id
            tid = Threads.threadid()
            # Storage for gradient terms
            grad_log_q = model.storage_L2_par[tid]
            grad_L = model.storage_L3_par[tid]
            # Storage for intermediate term in gradient calculations
            D_beta = model.storage_L_par[tid]
            rho_star_old_i = view(model.storage_LL3_par[tid], 1:L)
            # Get parameters for variational distribution of skill of i-th student
            pi_star_old_i = pi_star_old[i][1]
            # Get group number of student i
            group_i = obs.group[i]
            # Perform gradient descent update of i-th π*    
            @inbounds for iter in 1:maxiter
                # Rho is unique up to a constant addative term
                rho_star_old_i = log.(pi_star_old_i)
                # Sample Z with updated π*
                sample_Z(model, i, 1)
                # Set gradient of ELBO to 0
                fill!(grad_L, 0)
                # Rao Blackwellized ELBO
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                @inbounds for m in 1:M
                    z_im = Z_sample[i][1][m]
                    # Calculate gradient of log(q_1i(Z_i)) w.r.t. π*_i
                    grad_log_q .= z_im .- pi_star_old_i
                    # Calculate log(p(Y, Z_(i)))
                    log_prob_YZ = 0
                    for j in 1:J
                        mul!(D_beta, D[j], beta_sample[j][m])
                        log_prob_YZ += dot(z_im, log.(sigmoid.((2*Y[i, 1, j] - 1) .* D_beta)))
                    end
                    skill_profile = obs.skill_dict[argmax(z_im)]
                    for k in 1:K
                        log_prob_YZ += log(sigmoid((2*skill_profile[k] - 1) * dot(gamma_sample[k][1][1][group_i][m], obs.X[k][1][i])))
                    end
                    # Calculate log(q_1i(Z_i))
                    log_q = dot(z_im, log.(pi_star_old_i))
                    # Update average gradient
                    grad_L .= (m - 1)/m .* grad_L + 1/m .* grad_log_q .* (log_prob_YZ - log_q)
                    # Update ELBO estimator
                    ELBO = (m-1)/m * ELBO + 1/m * (log_prob_YZ - log_q)
                end
                # Print ELBO, parameter and gradient if verbose
                if verbose
                    println("ELBO: $ELBO")
                    println("π*_$i: $pi_star_old_i")
                    println("gradient: $grad_L")
                end
                # Update with one step
                rho_star_old_i .+= step * grad_L
                # Convert logits into probabilities
                pi_star_old_i .= exp.(rho_star_old_i) ./ sum(exp.(rho_star_old_i))
                # Stop condition
                if abs2(norm(grad_L)) <= tol
                    break
                end
            end
        end
    end
end

update_categorical_variational_distribution (generic function with 2 methods)

In [51]:
function update_categorical_variational_distribution2(
    model               :: TDCModel,
    time                :: Int;
    step                :: T = 1e-2,
    tol                 :: T = 1e-6,
    maxiter             :: Int = 100000,
    verbose             :: Bool = true
) where T <: AbstractFloat
    obs = model.obs
    Y, D, X = obs.Y, obs.D, obs.X
    Z_sample, beta_sample, gamma_sample = model.Z_sample, model.beta_sample, model.gamma_sample
    pi_star_old = model.pi_star
    # Number of students, time points, questions, skills, attribute profiles, groups
    N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)
    M = model.M
    # Fully update parameters of each Z_i using noisy gradients before moving to update parameters of next Z_i
    if !model.enable_parallel
        @inbounds for i in 1:N
            # Storage for gradient terms
            grad_log_q = model.storage_L2
            grad_L = model.storage_L3
            # Storage for intermediate term in gradient calculations
            D_beta = model.storage_L
            rho_star_old_i = view(model.storage_LL3, 1:L)
            # Get parameters for variational distribution of skill of i-th student
            pi_star_old_i = pi_star_old[i][time]
            # Get group number of student i
            group_i = obs.group[i]
            # Perform gradient descent update of i-th π*    
            for iter in 1:maxiter
                # Rho is unique up to a constant addative term
                rho_star_old_i = log.(pi_star_old_i)
                # Sample Z with updated π*
                sample_Z(model, i, time)
                # Set gradient of ELBO to 0
                fill!(grad_L, 0)
                # Rao Blackwellized ELBO
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                for m in 1:M
                    z_im = Z_sample[i][time][m]
                    # Calculate gradient of log(q_1i(Z_i)) w.r.t. π*_i
                    grad_log_q .= z_im .- pi_star_old_i
                    # Calculate log(p(Y, Z_(i)))
                    log_prob_YZ = 0
                    for j in 1:J
                        mul!(D_beta, D[j], beta_sample[j][m])
                        log_prob_YZ += dot(z_im, log.(sigmoid.((2*Y[i, time, j] - 1) .* D_beta)))
                    end
                    skill_profile = obs.skill_dict[argmax(z_im)]
                    prev_skill_profile = obs.skill_dict[argmax(Z_sample[i][time - 1][m])]
                    for k in 1:K
                        log_prob_YZ += log(sigmoid((2*skill_profile[k] - 1) * 
                                    dot(gamma_sample[k][time][prev_skill_profile[k] + 1][group_i][m], obs.X[k][time][i, :])))
                    end
                    # Calculate log(q_1i(Z_i))
                    log_q = dot(z_im, log.(pi_star_old_i))
                    # Update average gradient
                    grad_L .= (m - 1)/m .* grad_L + 1/m .* grad_log_q .* (log_prob_YZ - log_q)
                    # Update ELBO estimator
                    ELBO = (m-1)/m * ELBO + 1/m * (log_prob_YZ - log_q)
                end
                # Print ELBO, parameter and gradient if verbose
                if verbose
                    println("ELBO: $ELBO")
                    println("π*_$i: $pi_star_old_i")
                    println("gradient: $grad_L")
                end
                # Update with one step
                rho_star_old_i .+= step * grad_L
                # Convert logits into probabilities
                pi_star_old_i .= exp.(rho_star_old_i) ./ sum(exp.(rho_star_old_i))
            end
        end
    else
        Threads.@threads for i in 1:N
            # Get thread id
            tid = Threads.threadid()
            # Storage for gradient terms
            grad_log_q = model.storage_L2_par[tid]
            grad_L = model.storage_L3_par[tid]
            # Storage for intermediate term in gradient calculations
            D_beta = model.storage_L_par[tid]
            rho_star_old_i = view(model.storage_LL3_par[tid], 1:L)
            # Get parameters for variational distribution of skill of i-th student
            pi_star_old_i = pi_star_old[i][time]
            # Get group number of student i
            group_i = obs.group[i]
            # Perform gradient descent update of i-th π*    
            @inbounds for iter in 1:maxiter
                # Rho is unique up to a constant addative term
                rho_star_old_i = log.(pi_star_old_i)
                # Sample Z with updated π*
                sample_Z(model, i, time)
                # Set gradient of ELBO to 0
                fill!(grad_L, 0)
                # Rao Blackwellized ELBO
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                for m in 1:M
                    z_im = Z_sample[i][time][m]
                    # Calculate gradient of log(q_1i(Z_i)) w.r.t. π*_i
                    grad_log_q .= z_im .- pi_star_old_i
                    # Calculate log(p(Y, Z_(i)))
                    log_prob_YZ = 0
                    for j in 1:J
                        mul!(D_beta, D[j], beta_sample[j][m])
                        log_prob_YZ += dot(z_im, log.(sigmoid.((2*Y[i, time, j] - 1) .* D_beta)))
                    end
                    skill_profile = obs.skill_dict[argmax(z_im)]
                    prev_skill_profile = obs.skill_dict[argmax(Z_sample[i][time - 1][m])]
                    for k in 1:K
                        log_prob_YZ += log(sigmoid((2*skill_profile[k] - 1) * 
                                    dot(gamma_sample[k][time][prev_skill_profile[k] + 1][group_i][m], obs.X[k][time][i, :])))
                    end
                    # Calculate log(q_1i(Z_i))
                    log_q = dot(z_im, log.(pi_star_old_i))
                    # Update average gradient
                    grad_L .= (m - 1)/m .* grad_L + 1/m .* grad_log_q .* (log_prob_YZ - log_q)
                    # Update ELBO estimator
                    ELBO = (m-1)/m * ELBO + 1/m * (log_prob_YZ - log_q)
                end
                # Print ELBO, parameter and gradient if verbose
                if verbose
                    println("ELBO: $ELBO")
                    println("π*_$i: $pi_star_old_i")
                    println("gradient: $grad_L")
                end
                # Update with one step
                rho_star_old_i .+= step * grad_L
                # Convert logits into probabilities
                pi_star_old_i .= exp.(rho_star_old_i) ./ sum(exp.(rho_star_old_i))
            end
        end
    end
end

update_categorical_variational_distribution2 (generic function with 1 method)

In [ ]:
function update_normal_variational_distribution(
    model       :: TDCModel;
    init_step   :: T=1e-3,
    step_iterator=get_robbins_monroe_iterator(init_step, 20),
    # step_iterator_factory=get_robbins_monroe_iterator,
    use_iter    :: Bool=false,
    tol         :: T=1e-6,
    maxiter     :: Int=100000,
    verbose     :: Bool=true
) where T <: AbstractFloat
    obs = model.obs
    Y, D = Matrix{T}(obs.Y), Vector{Matrix{T}}(obs.D)
    Z_sample = model.Z_sample
    mu_star_old, V_star_old = model.mu_star, model.V_star
    N, J, L = size(Y, 1), size(Y, 2), size(D[1], 1)
    M = model.M
    # Sample Z, β, and sigma^2. Only β samples will update as the parameters update
    sample_variational_distribution(model, sample_Z=true, sample_sigma2=true)
    # Fully update parameters of each β_j using noisy gradients before moving to update parameters of next β_j
    if !model.enable_parallel
        @inbounds for j in 1:J
            mu_star_old_j = mu_star_old[j]
            V_star_old_j = V_star_old[j]
            # Perform gradient descent update of mu_j and V_j
            len_beta = length(beta_sample[j][1])
            # Assign storage for gradient terms
            # Memory assigned from preallocated storage
            # Memory has to be strided (equal stride between memory addresses) to work with BLAS and LAPACK 
            # (important for vectorized matricies to be strided if we want to use them for linear algebra)
            # Matricies are stored column major in Julia, so memory is assigned by column left to right
            grad_mu_L = view(model.storage_L, 1:len_beta)
            grad_C_L = view(model.storage_LL2, 1:len_beta, 1:len_beta)
            vech_grad_C_L = view(grad_C_L, [len_beta * (j - 1) + i for j in 1:len_beta for i in j:len_beta]) # Uses same memory as grad_C_L
            grad_mu_log_q = view(model.storage_L2, 1:len_beta)
            vec_grad_V_log_q = view(model.storage_LL3, 1:len_beta^2)
            grad_V_log_q = reshape(vec_grad_V_log_q, len_beta, len_beta) # Uses same memory as vec_grad_V_log_q
            # Assign storage for calculating intermediate terms for gradient
            Vinv_star_old_j = view(model.storage_LL, 1:len_beta, 1:len_beta)
            beta_minus_mu = view(model.storage_L3, 1:len_beta)
            C_star_old_j = view(model.storage_C, 1:len_beta, 1:len_beta)
            vech_C_star_old_j = view(C_star_old_j, [len_beta * (j - 1) + i for j in 1:len_beta for i in j:len_beta]) # Uses same memory as C_star_old_j
            fill!(C_star_old_j, 0)
            storage_kron_prod = view(model.storage_L2L2, 1:len_beta^2, 1:len_beta^2)
            storage_len_beta_sqr = view(model.storage_Lsqr, 1:len_beta^2)
            storage_len_beta_sqr2 = view(model.storage_Lsqr2, 1:len_beta^2)
            storage_gradC = view(model.storage_gradC, 1:Int(len_beta * (len_beta + 1) / 2))
            # Generate commutation and duplication matrix
            comm_j = view(model.storage_comm, 1:len_beta^2, 1:len_beta^2)
            dup_j = view(model.storage_dup, 1:len_beta^2, 1:Int(len_beta * (len_beta + 1) / 2))
            get_comm!(comm_j, len_beta)
            get_dup!(dup_j, len_beta)
            # Assign len_beta by len_beta identity matrix
            I_j = view(model.I_LL, 1:len_beta, 1:len_beta)
            # # Get step size iterator
            # step_iterator = step_iterator_factory(init_step)
            # # Initialize variables for tracking previous values
            # prev_ELBO = -Inf
            # prev_mu = view(model.storage_L4, 1:len_beta)
            # prev_V = view(model.storage_LL4, 1:len_beta, 1:len_beta)
            # prev_mu .= mu_star_old[j]
            # prev_V .= V_star_old[j]
            @inbounds for iter in 1:maxiter
                # Sample β from variational distribution
                sample_variational_distribution(model, sample_β=true, idx_β=j)
                fill!(grad_mu_L, 0)
                fill!(grad_C_L, 0)
                # Copy V* into storage
                copy!(Vinv_star_old_j, V_star_old_j)
                # Perform cholesky decomposition on V*
                # After this step, the lower triangle of Vinv_star_old_j will contain the lower triangular cholesky factor of V*
                LAPACK.potrf!('L', Vinv_star_old_j)
                # Calculate log|V_j| from diagonal of cholesky decomposition
                logdet_V_j = 0
                for b in 1:len_beta
                    logdet_V_j += 2 * log(Vinv_star_old_j[b, b])
                end
                # Copy lower triangular cholesky factor into preallocated storage
                for k in 1:len_beta
                    for l in 1:k
                        C_star_old_j[k, l] = Vinv_star_old_j[k, l]
                    end
                end
                # Perform in place matrix inverse on positive definite V* matrix to get V* inverse
                LAPACK.potri!('L', Vinv_star_old_j)
                LinearAlgebra.copytri!(Vinv_star_old_j, 'L')
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                @inbounds for m in 1:M
                    beta_jm = beta_sample[j][m]
                    fill!(grad_mu_log_q, 0)
                    # grad_mu_log_q = Vinv_star * β_jm
                    BLAS.gemv!('N', T(1), Vinv_star_old_j, beta_jm, T(1), grad_mu_log_q)
                    # grad_mu_log_q = Vinv_star_j * β_jm - Vinv_star_j * mu_star_j
                    BLAS.gemv!('N', T(-1), Vinv_star_old_j, mu_star_old_j, T(1), grad_mu_log_q)
                    # grad_V_log_q = -1/2(Vinv_star_j - Vinv_star_j * (β_jm - mu_star_j) * (β_jm - mu_star_j)^T * Vinv_star_j)
                    copy!(grad_V_log_q, Vinv_star_old_j)
                    BLAS.gemm!('N', 'T', T(1 / 2), grad_mu_log_q, grad_mu_log_q, T(-1 / 2), grad_V_log_q)
                    # storage_kron_prod = I ⊗ C_j
                    collect!(storage_kron_prod, kronecker(I_j, C_star_old_j))
                    # storage_len_beta_sqr = (I ⊗ C_j)'vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), storage_kron_prod, vec_grad_V_log_q, T(1), fill!(storage_len_beta_sqr, 0))
                    # storage_kron_prod = C_j ⊗ I
                    collect!(storage_kron_prod, kronecker(C_star_old_j, I_j))
                    # storage_len_beta_sqr2 = (C_j ⊗ I)'vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), storage_kron_prod, vec_grad_V_log_q, T(1), fill!(storage_len_beta_sqr2, 0))
                    # storage_len_beta_sqr2 = ((C_j ⊗ I)' + K'(I ⊗ C_j)')vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), comm_j, storage_len_beta_sqr, T(1), storage_len_beta_sqr2)
                    # storage_gradC = D'((C_j ⊗ I)' + K'(I ⊗ C_j)')vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), dup_j, storage_len_beta_sqr2, T(1), fill!(storage_gradC, 0))
                    # Calculate log(p(Y, β_(j)))
                    log_prob_Ybeta = 0
                    for i in 1:N
                        fill!(model.storage_L3, 0)
                        BLAS.gemv!('N', (2 * Y[i, j] - 1), D[j], beta_jm, T(1), model.storage_L3)
                        log_prob_Ybeta += dot(Z_sample[i][m], log.(sigmoid.(model.storage_L3)))
                    end
                    log_prob_Ybeta -= 1 / (2 * sigma2_sample[m]) * dot(beta_jm, beta_jm)
                    beta_minus_mu .= beta_jm
                    beta_minus_mu .-= mu_star_old_j
                    log_q = -len_beta / 2 * log(2 * pi) - 1 / 2 * logdet_V_j - 1 / 2 * dot(beta_minus_mu, grad_mu_log_q)
                    # Update average gradient
                    grad_mu_L .= (m - 1) / m .* grad_mu_L + 1 / m .* grad_mu_log_q .* (log_prob_Ybeta - log_q)
                    vech_grad_C_L .= (m - 1) / m .* vech_grad_C_L + 1 / m .* storage_gradC .* (log_prob_Ybeta - log_q)
                    # Update ELBO estimator
                    ELBO = (m - 1) / m * ELBO + 1 / m * (log_prob_Ybeta - log_q)
                end
                # Print ELBO, parameter and gradient if verbose
                # if verbose
                #     println("ELBO: $ELBO")
                #     println("mu*_$j: $mu_star_old_j")
                #     println("gradient mu: $grad_mu_L")
                #     println("C*_$j: $C_star_old_j")
                #     println("gradient C: $grad_C_L")
                # end
                # Stop condition TODO: update to more appropriate stop condition
                # if abs2(norm(vech_C_star_old_j)) > 1e6
                #     break
                # end
                # # If ELBO decreases, go to previous step
                # if ELBO < prev_ELBO
                #     mu_star_old_j .= prev_mu
                #     V_star_old_j .= prev_V
                # else
                #     # Save current values
                #     prev_mu .= mu_star_old_j
                #     prev_V .= V_star_old_j
                #     prev_ELBO = ELBO
                #     # Update mu and C with one step
                #     mu_star_old_j .+= step .* grad_mu_L
                #     vech_C_star_old_j .+= step .* vech_grad_C_L
                #     # Set V_star_old_j = C * C'
                #     BLAS.gemm!('N', 'T', T(1), C_star_old_j, C_star_old_j, T(1), fill!(V_star_old_j, 0))
                # end
                # Update mu and C with one step
                step = init_step
                if use_iter
                    step = step_iterator()
                    println("Question $j: $step")
                end
                mu_star_old_j .+= step .* grad_mu_L ./ norm(grad_mu_L)
                vech_C_star_old_j .+= step .* vech_grad_C_L ./ norm(vech_grad_C_L)
                # Set V_star_old_j = C * C'
                BLAS.gemm!('N', 'T', T(1), C_star_old_j, C_star_old_j, T(1), fill!(V_star_old_j, 0))
            end
        end
    else
        Threads.@threads for j in 1:J
            # Get thread id
            tid = Threads.threadid()
            # Perform gradient descent update of mu_j and V_j
            len_beta = length(beta_sample[j][1])
            # Assign storage for gradient terms
            # Memory assigned from preallocated storage
            # Memory has to be strided (equal stride between memory addresses) to work with BLAS and LAPACK 
            # (important for vectorized matricies to be strided if we want to use them for linear algebra)
            # Matricies are stored column major in Julia, so memory is assigned by column left to right
            grad_mu_L = view(model.storage_L_par[tid], 1:len_beta)
            grad_C_L = view(model.storage_LL2_par[tid], 1:len_beta, 1:len_beta)
            vech_grad_C_L = view(grad_C_L, [len_beta * (j - 1) + i for j in 1:len_beta for i in j:len_beta]) # Uses same memory as grad_C_L
            grad_mu_log_q = view(model.storage_L2_par[tid], 1:len_beta)
            vec_grad_V_log_q = view(model.storage_LL3_par[tid], 1:len_beta^2)
            grad_V_log_q = reshape(vec_grad_V_log_q, len_beta, len_beta) # Uses same memory as vec_grad_V_log_q
            # Assign storage for calculating intermediate terms for gradient
            Vinv_star_old_j = view(model.storage_LL_par[tid], 1:len_beta, 1:len_beta)
            beta_minus_mu = view(model.storage_L3_par[tid], 1:len_beta)
            C_star_old_j = view(model.storage_C_par[tid], 1:len_beta, 1:len_beta)
            vech_C_star_old_j = view(C_star_old_j, [len_beta * (j - 1) + i for j in 1:len_beta for i in j:len_beta]) # Uses same memory as C_star_old_j
            fill!(C_star_old_j, 0)
            storage_kron_prod = view(model.storage_L2L2_par[tid], 1:len_beta^2, 1:len_beta^2)
            storage_len_beta_sqr = view(model.storage_Lsqr_par[tid], 1:len_beta^2)
            storage_len_beta_sqr2 = view(model.storage_Lsqr2_par[tid], 1:len_beta^2)
            storage_gradC = view(model.storage_gradC_par[tid], 1:Int(len_beta * (len_beta + 1) / 2))
            # Generate commutation and duplication matrix
            comm_j = view(model.storage_comm_par[tid], 1:len_beta^2, 1:len_beta^2)
            dup_j = view(model.storage_dup_par[tid], 1:len_beta^2, 1:Int(len_beta * (len_beta + 1) / 2))
            get_comm!(comm_j, len_beta)
            get_dup!(dup_j, len_beta)
            # Assign len_beta by len_beta identity matrix
            I_j = view(model.I_LL, 1:len_beta, 1:len_beta)
            # # Get step iterator
            # step_iterator = step_iterator_factory(init_step)
            # # Initialize variables for tracking previous values
            # prev_ELBO = -Inf
            # prev_mu = view(model.storage_L4, 1:len_beta)
            # prev_V = view(model.storage_LL4, 1:len_beta, 1:len_beta)
            # prev_mu .= mu_star_old[j]
            # prev_V .= V_star_old[j]
            @inbounds for iter in 1:maxiter
                # Sample β from variational distribution
                sample_variational_distribution(model, sample_β=true, idx_β=j)
                fill!(grad_mu_L, 0)
                fill!(grad_C_L, 0)
                mu_star_old_j = mu_star_old[j]
                V_star_old_j = V_star_old[j]
                # Copy V* into storage
                copy!(Vinv_star_old_j, V_star_old_j)
                # Perform cholesky decomposition on V*
                # After this step, the lower triangle of Vinv_star_old_j will contain the lower triangular cholesky factor of V*
                LAPACK.potrf!('L', Vinv_star_old_j)
                # Calculate log|V_j| from diagonal of cholesky decomposition
                logdet_V_j = 0
                for b in 1:len_beta
                    logdet_V_j += 2 * log(Vinv_star_old_j[b, b])
                end
                # Copy lower triangular cholesky factor into preallocated storage
                for k in 1:len_beta
                    for l in 1:k
                        C_star_old_j[k, l] = Vinv_star_old_j[k, l]
                    end
                end
                # Perform in place matrix inverse on positive definite V* matrix to get V* inverse
                LAPACK.potri!('L', Vinv_star_old_j)
                LinearAlgebra.copytri!(Vinv_star_old_j, 'L')
                ELBO = 0
                # Calculate the gradient estimate of the m-th sample
                @inbounds for m in 1:M
                    beta_jm = beta_sample[j][m]
                    fill!(grad_mu_log_q, 0)
                    # grad_mu_log_q = Vinv_star * β_jm
                    BLAS.gemv!('N', T(1), Vinv_star_old_j, beta_jm, T(1), grad_mu_log_q)
                    # grad_mu_log_q = Vinv_star_j * β_jm - Vinv_star_j * mu_star_j
                    BLAS.gemv!('N', T(-1), Vinv_star_old_j, mu_star_old_j, T(1), grad_mu_log_q)
                    # grad_V_log_q = -1/2(Vinv_star_j - Vinv_star_j * (β_jm - mu_star_j) * (β_jm - mu_star_j)^T * Vinv_star_j)
                    copy!(grad_V_log_q, Vinv_star_old_j)
                    BLAS.gemm!('N', 'T', T(1 / 2), grad_mu_log_q, grad_mu_log_q, T(-1 / 2), grad_V_log_q)
                    # storage_kron_prod = I ⊗ C_j
                    collect!(storage_kron_prod, kronecker(I_j, C_star_old_j))
                    # storage_len_beta_sqr = (I ⊗ C_j)'vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), storage_kron_prod, vec_grad_V_log_q, T(1), fill!(storage_len_beta_sqr, 0))
                    # storage_kron_prod = C_j ⊗ I
                    collect!(storage_kron_prod, kronecker(C_star_old_j, I_j))
                    # storage_len_beta_sqr2 = (C_j ⊗ I)'vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), storage_kron_prod, vec_grad_V_log_q, T(1), fill!(storage_len_beta_sqr2, 0))
                    # storage_len_beta_sqr2 = ((C_j ⊗ I)' + K'(I ⊗ C_j)')vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), comm_j, storage_len_beta_sqr, T(1), storage_len_beta_sqr2)
                    # storage_gradC = D'((C_j ⊗ I)' + K'(I ⊗ C_j)')vec(grad_V_log_q)
                    BLAS.gemv!('T', T(1), dup_j, storage_len_beta_sqr2, T(1), fill!(storage_gradC, 0))
                    # Calculate log(p(Y, β_(j)))
                    log_prob_Ybeta = 0
                    for i in 1:N
                        fill!(model.storage_L3_par[tid], 0)
                        BLAS.gemv!('N', (2 * Y[i, j] - 1), D[j], beta_jm, T(1), model.storage_L3_par[tid])
                        log_prob_Ybeta += dot(Z_sample[i][m], log.(sigmoid.(model.storage_L3_par[tid])))
                    end
                    log_prob_Ybeta -= 1 / (2 * sigma2_sample[m]) * dot(beta_jm, beta_jm)
                    beta_minus_mu .= beta_jm
                    beta_minus_mu .-= mu_star_old_j
                    log_q = -len_beta / 2 * log(2 * pi) - 1 / 2 * logdet_V_j - 1 / 2 * dot(beta_minus_mu, grad_mu_log_q)
                    # Update average gradient
                    grad_mu_L .= (m - 1) / m .* grad_mu_L + 1 / m .* grad_mu_log_q .* (log_prob_Ybeta - log_q)
                    vech_grad_C_L .= (m - 1) / m .* vech_grad_C_L + 1 / m .* storage_gradC .* (log_prob_Ybeta - log_q)
                    # Update ELBO estimator
                    ELBO = (m - 1) / m * ELBO + 1 / m * (log_prob_Ybeta - log_q)
                end
                step = init_step
                if use_iter
                    step = step_iterator()
                end
                mu_star_old_j .+= step .* grad_mu_L ./ norm(grad_mu_L)
                vech_C_star_old_j .+= step .* vech_grad_C_L ./ norm(vech_grad_C_L)
                # Set V_star_old_j = C * C'
                BLAS.gemm!('N', 'T', T(1), C_star_old_j, C_star_old_j, T(1), fill!(V_star_old_j, 0))
            end
        end
    end
end

In [52]:
using RCall

R"""
load("TDCM_multilevel_data.RData")
"""
TDCM_data = @rget data
Y = Array{Int, 3}(TDCM_data[:Y])
Q = convert(Matrix{Int64}, TDCM_data[:Q_matrix])
U = Vector{Vector{Matrix{Float64}}}(TDCM_data[:X_group])
for skill in TDCM_data[:X_ind]
    for time in 1:length(skill)
        if skill[time] isa Vector{<: Number}
            skill[time] = reshape(skill[time], :, 1)
        end
    end
end
X = Vector{Vector{Matrix{Float64}}}(TDCM_data[:X_ind])
group = Vector{Int64}(TDCM_data[:group])
obs = TDCMObs(Y, Q, U, X, group)

# R"""
# load("TDCM_multilevel_J50_data.RData")
# """
# TDCM_J50_data = @rget data_large_questions
# Y = Array{Int, 3}(TDCM_J50_data[:Y])
# Q = convert(Matrix{Int64}, TDCM_J50_data[:Q_matrix])
# U = Vector{Vector{Matrix{Float64}}}(TDCM_J50_data[:X_group])
# for skill in TDCM_J50_data[:X_ind]
#     for time in 1:length(skill)
#         if skill[time] isa Vector{<: Number}
#             skill[time] = reshape(skill[time], :, 1)
#         end
#     end
# end
# X = Vector{Vector{Matrix{Float64}}}(TDCM_J50_data[:X_ind])
# group = Vector{Int64}(TDCM_J50_data[:group])
# obs_J50 = TDCMObs(Y, Q, U, X, group)
;

In [56]:
N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)

mu_beta_prior = Vector{Vector{Float64}}(undef, J)
V_beta_prior = Vector{Matrix{Float64}}(undef, J)
for j in 1:J
    num_features = size(obs.D[j], 2)
    mu_beta_prior[j] = zeros(num_features)
    V_beta_prior[j] = Matrix(1.0I, num_features, num_features)
end

mu_omega_prior = Vector{Vector{Vector{Vector{Vector{Float64}}}}}(undef, K)
V_omega_prior = Vector{Vector{Vector{Vector{Matrix{Float64}}}}}(undef, K)
a_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)
b_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)

for k in 1:K
    mu_omega_prior[k] = Vector{Vector{Vector{Vector{Float64}}}}(undef, O)
    V_omega_prior[k] = Vector{Vector{Vector{Matrix{Float64}}}}(undef, O)
    a_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    b_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    for t in 1:O
        num_features_gamma = size(obs.X[k][t], 2)
        num_features_omega = size(obs.U[k][t], 2)
        if t == 1
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 1)
            mu_omega_prior[k][t][1] = Vector{Vector{Float64}}(undef, 1)
            mu_omega_prior[k][t][1][1] = zeros(num_features_omega)

            V_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 1)
            V_omega_prior[k][t][1] = Vector{Matrix{Float64}}(undef, 1)
            V_omega_prior[k][t][1][1] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)

            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            a_tau_prior[k][t][1] = ones(1)

            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            b_tau_prior[k][t][1] = ones(1)
        else
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 2)
            V_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 2)
            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            for z in 1:2
                mu_omega_prior[k][t][z] = Vector{Vector{Float64}}(undef, num_features_gamma)
                V_omega_prior[k][t][z] = Vector{Matrix{Float64}}(undef, num_features_gamma)
                a_tau_prior[k][t][z] = ones(num_features_gamma) .* 3
                b_tau_prior[k][t][z] = ones(num_features_gamma) .* 3
                for m in 1:num_features_gamma
                    mu_omega_prior[k][t][z][m] = zeros(num_features_omega)
                    V_omega_prior[k][t][z][m] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)
                end
            end
        end
    end
end

M = 100
model = TDCModel(obs, mu_beta_prior, V_beta_prior, mu_omega_prior, V_omega_prior, a_tau_prior, b_tau_prior, M, enable_parallel=true)

# J50 = size(obs_J50.Y, 3)
# mu_beta_prior = Vector{Vector{Float64}}(undef, J50)
# V_beta_prior = Vector{Matrix{Float64}}(undef, J50)
# for j in 1:J50
#     num_features = size(obs_J50.D[j], 2)
#     mu_beta_prior[j] = zeros(num_features)
#     V_beta_prior[j] = Matrix(1.0I, num_features, num_features)
# end
# model_J50 = TDCModel(obs_J50, mu_beta_prior, V_beta_prior, mu_omega_prior, V_omega_prior, a_tau_prior, b_tau_prior, M, enable_parallel=false)
;

TDCModel constructed for computation on 12 threads


In [57]:
# @resumable pi_t1_loglike = function(Z :: Vector{Int}, Y, :: Vector{Int}, D_beta :: Vector{AbstractFloat})
#     for m in 1:model.M
#         J = size(Y)
#         log_prob_YZ = 0
#         for j in 1:J
#             mul!(D_beta, D[j], beta_sample[j][m])
#             log_prob_YZ += dot(z_im, log.(sigmoid.((2*Y[i, 1, j] - 1) .* D_beta)))
#         end
#         skill_profile = obs.skill_dict[argmax(z_im)]
#         for k in 1:K
#             log_prob_YZ += log(sigmoid((2*skill_profile[k] - 1) * dot(gamma_sample[k][1][1][group_i][m], obs.X[k][1][i])))
#         end
#     end
# end

**Estimating attribute profiles from uniform initialization**

In [58]:
# Fix true values of gamma and betas
for k in 1:K
    for s in S
        model.mu_gamma_star[k][1][1][s] .= data[:gamma][k][1][s]
        model.V_gamma_star[k][1][1][s] = model.V_gamma_star[k][1][1][s] ./ 100
    end
end
for j in 1:J
    model.mu_beta_star[j] = data[:beta][j]
    model.V_beta_star[j] = model.V_beta_star[j] ./ 100
end

# Sample beta and gammas
sample_β(model)
for k in 1:K
    for t in 1:O
        if t == 1
            for s in 1:S
                sample_γ(model, s, t, k, 0)
            end
        else
            for z in 0:1
                for s in 1:S
                    sample_γ(model, s, t, k, z)
                end
            end
        end
    end
end

In [59]:
update_categorical_variational_distribution(model, maxiter=10, verbose=false)

In [60]:
update_categorical_variational_distribution2(model, 2, maxiter=10, verbose=false)

In [61]:
skill_profiles = Dict(1=>[0, 0], 
                 2=>[0, 1],
                 3=>[1, 0],
                 4=>[1, 1])

skill_numbers = Dict([0, 0]=>1, 
                 [0, 1]=>2,
                 [1, 0]=>3,
                 [1, 1]=>4)

preds = []
accuracy = 0
attribute_accs = zeros(L)
group_accs = zeros(S)

attribute_counts = zeros(L)
group_counts = zeros(S)

for i in 1:5000
    pred = skill_profiles[argmax(model.pi_star[i][1])]
    actual = Vector{Int}(data[:profiles][i, :, 1])
    group_i = obs.group[i]
    profile_i = skill_numbers[actual]
    push!(preds, pred)
    correct = all(pred .== actual)
    accuracy += correct
    attribute_accs[profile_i] += correct
    group_accs[group_i] += correct
    attribute_counts[profile_i] += 1
    group_counts[group_i] += 1
    # if !correct
    #     print(Vector{Int}(data[:profiles][i, 1, :]))
    #     println(model.pi_star[i][1])
    # end
end

accuracy = accuracy/N
attribute_accs .= attribute_accs ./ attribute_counts
group_accs .= group_accs ./ group_counts
;

In [62]:
attribute_accs

4-element Vector{Float64}:
 0.9962143273150844
 1.0
 0.9985815602836879
 1.0

In [63]:
preds = []
accuracy = 0
attribute_accs = zeros(L)
group_accs = zeros(S)

attribute_counts = zeros(L)
group_counts = zeros(S)

for i in 1:5000
    pred = skill_profiles[argmax(model.pi_star[i][2])]
    actual = Vector{Int}(data[:profiles][i, :, 2])
    group_i = obs.group[i]
    profile_i = skill_numbers[actual]
    push!(preds, pred)
    correct = all(pred .== actual)
    accuracy += correct
    attribute_accs[profile_i] += correct
    group_accs[group_i] += correct
    attribute_counts[profile_i] += 1
    group_counts[group_i] += 1
    # if !correct
    #     print(Vector{Int}(data[:profiles][i, 1, :]))
    #     println(model.pi_star[i][1])
    # end
end

accuracy = accuracy/N
attribute_accs .= attribute_accs ./ attribute_counts
group_accs .= group_accs ./ group_counts
;

In [64]:
attribute_accs

4-element Vector{Float64}:
 0.996538081107814
 0.9981818181818182
 0.9951298701298701
 0.9988962472406181